## Import Libraries

In [18]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

## Run Vivado

In [19]:
modules = os.listdir("Modules")
top_modules = [mod for mod in modules if mod[:3]=='top']
top_modules
os.system("vivado -mode batch -source Script/tcl_add.tcl")

0

In [20]:
os.mkdir('Impl')
for filename in top_modules:
    os.mkdir("Impl/"+filename[:-2])
    os.system("vivado -mode batch -source Script/tcl_run.tcl -tclargs {}".format(filename))

## Analyse Metric

In [122]:
designs = os.listdir("Impl")
file = ["power.txt", "timing.txt","utilization.txt"]
col = ["Signal Power(w)","Logic Power(w)","Static Power(w)","Dynamic Power(w)","Total Power(w)","DataPath Delay(ns)","F7_Muxes","F8_Muxes","LUT_as_Memory","LUT_as_Logic","Slice_LUTs","LUT2","LUT3","LUT4","LUT5","LUT6"]
os.mkdir('Result')     

In [123]:
sigPow = []
logPow = []
statPow = []
dcPow = []
totPow = []
datPthDly = []
F7Mux = []
F8Mux = []
LUTasMem = []
LUTasLog = []
sliceLUT = []
LUT2 = []
LUT3 = []
LUT4 = []
LUT5 = []
LUT6 = []

for des in designs:
    for fl in file:
        with open("Impl/{}/{}".format(des,fl)) as f:
            lines = f.read()
            
        sp = lines.split("\n")
        
        for l in sp:
            if fl == "power.txt":
                if "Signals" in l:
                    sigP = float(l.split()[3])
                    sigPow.append(sigP)
                if "Slice Logic" in l:
                    logP = float(l.split()[4])
                    logPow.append(logP)
                if "Static Power" in l:
                    statP = float(l.split()[4])
                    statPow.append(statP)
                if "Dynamic (W)" in l:
                    dcP = float(l.split()[4])
                    dcPow.append(dcP)
                if "Total On-Chip Power (W)" in l:
                    totP = float(l.split()[6])
                    totPow.append(totP)
                    
            if fl == "timing.txt":
                if "Data Path Delay" in l:
                    dPD = float(l.split()[3][:-2])
                    datPthDly.append(dPD)
            
            if fl == "utilization.txt": 
                if "F7 Muxes" in l:
                    valF7Mux = int(l.split()[4])
                    F7Mux.append(valF7Mux)
                if "F8 Muxes" in l:
                    valF8Mux = int(l.split()[4])
                    F8Mux.append(valF8Mux)
                if "LUT as Memory         | " in l:
                    valLUTasMem = int(l.split()[5])
                    LUTasMem.append(valLUTasMem)
                if "LUT as Logic          |" in l:
                    valLUTasLog = int(l.split()[5])
                    LUTasLog.append(valLUTasLog)
                if "Slice LUTs              |" in l:
                    valsliceLUT = int(l.split()[4])
                    sliceLUT.append(valsliceLUT)
                if "| LUT2     |" in l:
                    valLUT2 = int(l.split()[3])
                    LUT2.append(valLUT2)
                if "| LUT3     |" in l:
                    valLUT3 = int(l.split()[3])
                    LUT3.append(valLUT3)
                if "| LUT4     |" in l:
                    valLUT4 = int(l.split()[3])
                    LUT4.append(valLUT4)                    
                if "| LUT5     |" in l:
                    valLUT5 = int(l.split()[3])
                    LUT5.append(valLUT5)                    
                if "| LUT6     |" in l:
                    valLUT6 = int(l.split()[3])
                    LUT6.append(valLUT6)                

## Write Data to CSV File

In [135]:

result = [sigPow, logPow,statPow, dcPow, totPow, datPthDly, F7Mux, F8Mux, LUTasMem, LUTasLog, sliceLUT, LUT2, LUT3, LUT4, LUT5, LUT6]
df = pd.DataFrame(result) 
df.rows = designs
df.insert(loc=0, column='', value=col)
df = df.T
df.insert(loc=0, column='', value=["Module Name","top_CLA16","top_CLA32","top_CLA64","top_CLA8"])

df.to_csv("Result/Result.csv",index=False)

C:\Users\NITESH MAURYA\AppData\Local\Temp\ipykernel_7124\2769454819.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.rows = designs
